### Import packages

In [1]:
import numpy as np
from pypuf.simulation import *
import pypuf.metrics as pm
from pypuf.io import random_inputs
import pypuf.io, pypuf.simulation
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import metrics, activations
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.initializers import VarianceScaling
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from mmoe import MMoE
import datetime
import psutil
from tensorflow.python.profiler import profiler_client
from preprocessing import *

process = psutil.Process()
memory_info_start = process.memory_info()

# Memory cost
print(f"Memory: {memory_info_start.rss / (1024 ** 2):.2f} MB")


Memory: 265.58 MB


### Build the model

In [2]:

     
def Model_Multiple_PUFs(units_mmoe,gate_activation,N_train,train_c,test_c,train_r_groups,test_r_groups,PUF_list,num_expert=7):    
    num_features = 64
    input_layer = Input(shape=(num_features,))
    experts_list = []
    activation = 'relu'
    kernel_init = 'glorot_uniform'
    drop_out_rate = 0.2
    num_neu = 2**5

    # Build the PUF experts
    experts_list = [Expert_customize([num_neu,num_neu],activation=activation,kernel_init=kernel_init,drop_out_rate=drop_out_rate)
                    for _ in range(num_expert)]
    tower_units = [num_neu for _ in range(PUF_list.n_pufs)]


    # Set up MMoPE layer
    mmoe_layers = MMoE(
        units=units_mmoe,
        # num_experts=num_experts,
        use_gate_bias=True,
        experts=experts_list,
        num_tasks=PUF_list.n_pufs,
        expert_activation=activation,
        gate_activation=gate_activation,
        dropout_rate=0.1,
    )(input_layer)


    output_layers = []
    def lr_schedule(epoch, lr):
        if epoch < 30:
            return 0.01
        elif epoch < 60:
            return 0.005
        elif epoch < 300:
            return 0.001
        else:
            return 0.001
    # output_info = ['r1', 'r2', 'r3','r4','r5']
    output_info = ['r'+str(i) for i in range(1,PUF_list.n_pufs+1)]
    loss_dict = {'r'+str(i): 'binary_crossentropy' for i in range(1,PUF_list.n_pufs+1)}
    # output_info = ['r1', 'r2']
    train_losses = {output: [] for output in output_info}
    train_accuracies = {output: [] for output in output_info}
    val_losses = {output: [] for output in output_info}
    val_accuracies = {output: [] for output in output_info}

    # Custom callback to collect training history
    class HistoryCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            for output in output_info:
                train_losses[output].append(logs.get(f'{output}_binary_loss'))
                train_accuracies[output].append(logs.get(f'{output}_binary_accuracy'))
                val_losses[output].append(logs.get(f'val_{output}_binary_loss'))
                val_accuracies[output].append(logs.get(f'val_{output}_binary_accuracy'))

    # Build tower layer from MMoE layer
    for index, task_layer in enumerate(mmoe_layers):
        tower_layer = Dense(
            units=tower_units[index],
            activation=activation)(task_layer)
        output_layer = Dense(
            units=1,
            name=output_info[index],
            activation='sigmoid',
            kernel_initializer=kernel_init)(tower_layer)
        output_layers.append(output_layer)

    
    
    class LossWeightCallback(tf.keras.callbacks.Callback):
        def __init__(self, output_info,custom_layer, **kwargs):
            super(LossWeightCallback, self).__init__(**kwargs)
            self.output_info = output_info
            self.num_task = len(self.output_info)
            self.custom_layer = custom_layer
            self.dynamic_loss_weights = [K.variable(1.0) for _ in range(PUF_list.n_pufs)]
        def on_epoch_end(self, epoch, logs=None):
            logs = logs or {}
            Done = 0
            for i in range(self.num_task):
                current_accuracy = logs.get(self.output_info[i]+'_accuracy')
                # weight = 2 - 2*current_accuracy
                if current_accuracy>0.90:
                    K.set_value(self.custom_layer[i].threshold,0.01)
                    Done += 1
                    # weight=0.0
                # else:
                    weight = 2 - 2*current_accuracy
                    K.set_value(self.dynamic_loss_weights[i],weight)
                elif current_accuracy>0.8:
                    K.set_value(self.custom_layer[i].threshold,0.005)
                elif current_accuracy>0.7:
                    K.set_value(self.custom_layer[i].threshold,0.001)
                elif current_accuracy>0.6:
                    K.set_value(self.custom_layer[i].threshold,0.00001)
            if Done==self.num_task:
                self.model.stop_training = True
                print("Early Stop!")
                    # self.custom_layer[i].experts_needed = 7
    LossWeightUpdate = LossWeightCallback(output_info=output_info,custom_layer=gate_activation)
           
    # Compile model
    model = Model(inputs=[input_layer], outputs=output_layers)
    optimizer = 'adam'
    model.compile(
        loss='binary_crossentropy',
        optimizer = optimizer,
        loss_weights=LossWeightUpdate.dynamic_loss_weights,
        metrics=['accuracy']
    )

    model.summary()
    batch_size = min(N_train//40,20000)

    history = model.fit(
        x=train_c,
        y=train_r_groups,
        validation_data=(test_c, test_r_groups),
        batch_size=batch_size,
        epochs=1000
        ,callbacks = [LossWeightUpdate]
    )
    return model, history,output_info,train_losses,train_accuracies,val_losses,val_accuracies



### Generate data

In [3]:

import random
PUF_list = PUFs(stages=64, similarity=2)
PUF_list.seed = random.randint(1,1000)
CRP_seed = random.randint(1,100)
print("CRP seed=",CRP_seed)
print("seed=",PUF_list.seed)
N_train = 300000
# PUF_list.add_XOR_PUF(k=2,num=1)
PUF_list.add_XOR_PUF(k=4,num=1)
PUF_list.add_XOR_PUF(k=3,num=1)
# PUF_list.add_XOR_PUF(k=2,num=1)
PUF_list.add_XOR_PUF(k=5,num=1)
# PUF_list.add_FF_PUF([(32,50)],1)
# PUF_list.add_herero_XORFF_PUFs(k=2,ff=[[(20,50),(19,52)],[(21,54),(22,55)],[(30,60),(31,62)]],num=1)
# PUF_list.add_herero_XORFF_PUFs(k=3,ff=[[(20,50)],[(21,54)],[(30,60)]],num=1)
# PUF_list.add_interpose_PUFs(1,5,1)
# PUF_list.add_interpose_PUFs(2,2,1)
# PUF_list.add_interpose_PUFs(3,3,1)
# PUF_list.add_XORFF_PUF(2,[(20,50)],1)
# PUF_list.add_XORFF_PUF(4,[(20,50),(21,54)],1)
# PUF_list.add_XORFF_PUF(3,[(20,50),(21,54)],1)
# PUF_list.add_XORFF_PUF(2,[(21,54)],1)
# PUF_list.add_XORFF_PUF(2,[(21,54),(10,40)],1)
# PUF_list.add_XORFF_PUF(1,[(20,50),(10,40),(30,60)],1)
# PUF_list.add_XOR_PUF(1,num=1)

[c, responses] = PUF_list.generate_crps(CRP_seed,N_train)
c = get_parity_vectors2(c)
responses = np.array(responses)
train_c,test_c,train_r,test_r = train_test_split(c,responses.T,test_size=0.2, random_state=42)
train_r_groups = [train_r[:,i].reshape(-1,1) for i in range(PUF_list.n_pufs)]
test_r_groups = [test_r[:,i].reshape(-1,1) for i in range(PUF_list.n_pufs)]


CRP seed= 34
seed= 219


### Modelling three PUFs:
- 3-XOR APUF
- 4-XOR APUF
- 5-XOR APUF

In [5]:
units_mmoe = 32
model = None
import pandas as pd
######################################################################
# gate_activation = [CustomSoftmaxThre(threshold=1) for _ in range(PUF_list.n_pufs)]
## The change the total number of experts
num_experts = 20
gate_activation = [CustomSoftmaxExperts(experts_needed=5,threshold=0.00001) for _ in range(PUF_list.n_pufs)]
# output_info = ['r'+str(i) for i in range(1,PUF_list.n_pufs+1)]

# gate_activation = CustomSoftmaxExperts(experts_needed=5)
# gate_activation = [activations.get('softmax')]


model,history, output_info, train_losses, train_accuracies, val_losses, val_accuracies = Model_Multiple_PUFs(
    units_mmoe=units_mmoe,
    gate_activation=gate_activation,
    N_train=N_train,
    train_c=train_c,
    test_c=test_c,
    train_r_groups=train_r_groups,
    test_r_groups=test_r_groups,
    PUF_list=PUF_list,
    num_expert=num_experts
)

memory_info_end = process.memory_info() 

# Memory cost
print(f"Memory cost: {(memory_info_end.rss- memory_info_start.rss) / (1024 ** 3):.2f} GiB")

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
m_mo_e_1 (MMoE)                 [(None, 32), (None,  66626       input_2[0][0]                    
__________________________________________________________________________________________________
dense_83 (Dense)                (None, 32)           1056        m_mo_e_1[0][0]                   
__________________________________________________________________________________________________
dense_84 (Dense)                (None, 32)           1056        m_mo_e_1[0][1]                   
____________________________________________________________________________________________

Here we provide the demo with "early stop at 90\%", to get the results from the paper, please set the stop accuracy to 95\%.